In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import itertools

%matplotlib inline

### Load survey data

In [2]:
responses = pd.read_csv('DIAG Project Preference Survey (Responses) - Form Responses 1.csv')
responses['Timestamp'] = pd.to_datetime(responses['Timestamp'])

### Drop duplicates, keeping only the most recent submission

In [3]:
responses.sort_values('Timestamp', inplace=True)
responses.drop_duplicates(subset=['Username'], keep='last', inplace=True)
responses.index = range(len(responses))

### Load data about individuals and merge with form submission

In [4]:
peopleTypes = pd.read_csv('diagPeopleTypes.csv')
peopleTypes = peopleTypes[['Username', 'Technical Y=1, N=0', 'MBA 1=0, 2=1']]

In [5]:
responses = pd.merge(responses, peopleTypes, on='Username')

### Generate Teams

All projects get six team members except UMMS which gets 5. We want to have a minimum of 2 "Technical" people per team and one MBA2

In [6]:
projects = ['Rate your interest in UMMS', 
            'Rate your interest in Amcor',
            'Rate your interest in Gallagher Basset Waypoint:',
            'Rate your interest in Ford',
            'Rate your interest in Gallagher Basset WC']

teams = []
teams.extend(5*[0] + 6*[1] + 6*[2] + 6*[3] + 6*[4])

#Weights come from survey responses
weights = np.empty((responses.shape[0], len(projects)))
for k in range(len(projects)):
    weights[:,k] = responses[projects[k]].values

#If someone didn't submit a response for a project, set it to zero.
#Alternatively we could assume they are agnostic and set it to the mean
weights = np.nan_to_num(weights)

#Tech and year are just binary weights
tech = np.array(responses['Technical Y=1, N=0'].values)
year = np.array(responses['MBA 1=0, 2=1'].values)

#### Generate a bunch of cases randomly

Alternatively, we could generate all know solutions but I am lazy. 1e7 in total should do the trick

In [11]:
numMembers = len(responses)
numCases = 10000000
cases = np.zeros((len(projects), len(responses), numCases))
results = np.zeros((numCases, 40))

for i in range(numCases):
    members = range(len(responses))
    np.random.shuffle(members)
    
    for j in range(numMembers):
        cases[teams[j],members[j], i] = 1

    for j in range(len(projects)):
        results[i,:numMembers] += cases[j,:,i]*weights[:,j]
        results[i,numMembers+1+j] = np.dot(cases[j,:,i], tech)
        results[i,numMembers+6+j] = np.dot(cases[j,:,i], year)
        
    results[i,numMembers] = results[i,:numMembers].sum()
        
print 'Done'

Done


Convert results into a dataframe and create summary of number of "Technical" members and MBA2's per team

In [12]:
colNames = []
for i in range(numMembers):
    colNames.append(responses.loc[i,'Username'])

colNames.append('Satisfaction')

for var in ['Tech', 'Year']:
    for i in range(len(projects)):
        colNames.append('Team_%d_' % i + var)

resultsDF = pd.DataFrame(results, columns=colNames)
resultsDF['Min Ind Satisfaction'] = resultsDF[resultsDF.columns[:numMembers]].min(axis=1)
resultsDF['Tech Summary'] = resultsDF[colNames[1:len(projects)+1]].min(axis=1)
resultsDF['Year Summary'] = resultsDF[colNames[len(projects)+1:]].min(axis=1)

Pick out valid solutions

In [20]:
validResults = resultsDF.loc[(resultsDF['Min Ind Satisfaction'] > 1) & (resultsDF['Tech Summary'] >= 2) & (resultsDF['Year Summary'] >= 1)]
validResults = validResults.sort_values(by='Satisfaction', ascending=False)
print "Found %d valid solutions" % len(validResults)

Found 39 valid solutions


In [15]:
validResults['Satisfaction'].head()

3274316    107.0
3984564    106.0
1063210    106.0
1480593    105.0
6175376    104.0
Name: Satisfaction, dtype: float64

Arbitrarily take the top of the list as the solution

In [32]:
winner = cases[:,:,validResults.index[0]]
selectedDF = {}
selectedTeams = {}
for team in range(len(projects)):
    selectedDF[projects[team]] = []
    selectedTeams[projects[team]] = []
    for i in range(len(responses)):
        if winner[team,i] > 0:
            selectedDF[projects[team]].append(responses.loc[i, 'Username'])
            selectedTeams[projects[team]].append([responses.loc[i, 'Username'], weights[i,team]])
            
selectedDF = pd.DataFrame.from_dict(selectedDF, orient='index')
selectedDF.to_csv('selectedTeams.csv')